In [39]:
import gc
import numpy as np
import pandas as pd
from utils.logging_setup import init_logging, get_logger

init_logging(level="INFO")  # "INFO" o "DEBUG"
log = get_logger("notebook")

log.info("Notebook listo.")
log.debug("Mensaje solo visible si level es DEBUG.")

from utils.OpenDataTools import OpenDataTools

[notebook] - [INFO] - Notebook listo.


In [2]:
# Se inicializa el buscador de datos abiertos
open_data_tools = OpenDataTools()

[root] - [WARNING] - Requests made without an app_token will be subject to strict throttling limits.


In [3]:
# Los headers están en la segunda fila del archivo
matriz = pd.read_excel('/app/data/20260204_Plantilla_Matriz_de_Contratacion - copia.xlsx', sheet_name='Matriz', header=1)

In [4]:
matriz.columns

Index(['REGIONAL', 'CENTRO ZONAL', 'MUNICIPIO', 'Componente para la UDS',
       'SERVICIO SIM', 'NOMBRE DEL SERVICIO', 'ZONA 2026.', 'ATENCIONES',
       'UDS_Zona', 'Cupos_Zona', 'UDS', 'Meses', 'Forma de Contratación',
       'NIT CONTRATISTA 2026', 'CONTRATISTA 2026',
       'Es Asociacion de Padres y Madres Usuarias del Servicio', 'Fecha_FIn',
       'Valor 2025', 'Valor 2026', 'Aporte ICBF',
       'VALOR INICIAL OTROS CONCEPTOS\n$',
       'CANTIDAD ATENCIONES QUE \nAPLICA TASAS COMPENSATORIAS\n(HI)',
       'VR. TASAS COMPENSATORIAS (SOLO APLICA PARA HOGARES INFANTILES )\n$',
       'VALOR TOTAL INICIAL APORTE ICBF POR SERVICIO\n$',
       'VALOR TOTAL INICIAL APORTE ICBF POR CONTRATO (REGISTRO UNICO)\n$',
       '% APORTE EAS CONTRAPARTIDA', 'VALOR  CONTRAPARTIDA APORTE EAS\n$',
       'VALOR TOTAL INICAL DEL APORTE DE LA EAS (REGISTRO UNICO)\n$',
       'VALOR TOTAL DEL CONTRATO (UNICO REGISTRO)\n$',
       'VALOR TOTAL DEL CONTRATO (UNICO REGISTRO) EN SMLV',
       'ALERTA V

In [5]:
nits = matriz[~matriz['NIT CONTRATISTA 2026'].isna()]['NIT CONTRATISTA 2026'].unique()
names_workers = matriz[~matriz['CONTRATISTA 2026'].isna()]['CONTRATISTA 2026'].unique()

print('nits:', len(nits))
print('names_workers:', len(names_workers))

nits: 1350
names_workers: 1276


In [6]:
def get_order_identifiers_as_list(order_identificators_list: list, chunk_size: int = 1000):
    chunks = [order_identificators_list[i:i + chunk_size] for i in range(0, len(order_identificators_list), chunk_size)]
    return ["(" + ", ".join([f'"{elem}"' for elem in chunk]) + ")" for chunk in chunks]

In [13]:
nits_as_str = get_order_identifiers_as_list(nits, chunk_size=100)

In [15]:
contracts_by_nit = []
for nits_chain in nits_as_str:
    contracts_by_nit.append(open_data_tools.get_contratos_por_cedulas(nits_chain, limit=1000000))

Se identificaron 539 registros en SECOP I.
Se identificaron 1744 registros en SECOP II.
Se identificaron 594 registros en SECOP I.
Se identificaron 1620 registros en SECOP II.
Se identificaron 1249 registros en SECOP I.
Se identificaron 3110 registros en SECOP II.
Se identificaron 436 registros en SECOP I.
Se identificaron 1170 registros en SECOP II.
Se identificaron 1033 registros en SECOP I.
Se identificaron 2016 registros en SECOP II.
Se identificaron 623 registros en SECOP I.
Se identificaron 1244 registros en SECOP II.
Se identificaron 328 registros en SECOP I.
Se identificaron 967 registros en SECOP II.
Se identificaron 495 registros en SECOP I.
Se identificaron 1415 registros en SECOP II.
Se identificaron 586 registros en SECOP I.
Se identificaron 1523 registros en SECOP II.
Se identificaron 692 registros en SECOP I.
Se identificaron 1427 registros en SECOP II.
Se identificaron 810 registros en SECOP I.
Se identificaron 1140 registros en SECOP II.
Se identificaron 735 registros 

In [16]:
secopi = []
secopii = []

for group_result in contracts_by_nit:
    # Specific filters:
    # SECOP I
    group_result['SECOP_I']['anno_cargue_secop'] = group_result['SECOP_I']['anno_cargue_secop'].astype(int)
    group_result['SECOP_I'] = group_result['SECOP_I'][group_result['SECOP_I']['nit_de_la_entidad'].str.contains('89999923')]
    secopi.append(group_result['SECOP_I'][group_result['SECOP_I']['anno_cargue_secop'] >= 2024])
    # SECOP II
    group_result['SECOP_II'] = group_result['SECOP_II'][group_result['SECOP_II']['nit_entidad'].str.contains('89999923')]
    secopii.append(group_result['SECOP_II'][group_result['SECOP_II']['fecha_de_firma'] >= '2024-01-01'])

secopi = pd.concat(secopi)
secopii = pd.concat(secopii)

In [29]:
print('matriz:', matriz.shape)
print('secopi:', secopi.shape)
print('secopii:', secopii.shape)

matriz: (4187, 34)
secopi: (3, 26)
secopii: (7608, 18)


In [24]:
matriz['NIT CONTRATISTA 2026'] = matriz['NIT CONTRATISTA 2026'].astype(str)

In [31]:
secopii.columns

Index(['id_contrato', 'fecha_de_firma', 'tipo_de_contrato',
       'modalidad_de_contratacion', 'estado_contrato', 'objeto_del_contrato',
       'tipodocproveedor', 'documento_proveedor', 'proveedor_adjudicado',
       'tipo_de_identificaci_n_representante_legal',
       'identificaci_n_representante_legal', 'nombre_representante_legal',
       'nombre_entidad', 'nit_entidad', 'departamento', 'ciudad',
       'valor_del_contrato', 'urlproceso'],
      dtype='object')

In [ ]:
# documento_proveedor
# identificaci_n_representante_legal
merge = pd.merge(left=matriz, right=secopii, how='left', left_on='NIT CONTRATISTA 2026', right_on='documento_proveedor', indicator=True)

In [45]:
import pandas as pd
import numpy as np

pct = 0.03  # 3% tolerancia

valor_l = 'VALOR TOTAL DEL CONTRATO (UNICO REGISTRO)\n$'
valor_r = 'valor_del_contrato'
texto_l = 'NIT CONTRATISTA 2026'
texto_r = 'documento_proveedor'

# --- 0) ids para controlar 1-1 ---
L = matriz.copy()
R = secopii.copy()
L["_id_l"] = np.arange(len(L))
R["_id_r"] = np.arange(len(R))

# --- 1) números ---
L[valor_l] = pd.to_numeric(L[valor_l], errors="coerce")
R[valor_r] = pd.to_numeric(R[valor_r], errors="coerce")

L = L.dropna(subset=[valor_l]).sort_values(valor_l).copy()
R = R.dropna(subset=[valor_r]).sort_values(valor_r).copy()

# --- 2) candidato por cercanía ---
cand = pd.merge_asof(
    L,
    R,
    left_on=valor_l,
    right_on=valor_r,
    direction="nearest"
)

# --- 3) texto: normaliza a dígitos (NIT/documento) ---
def only_digits(x):
    if pd.isna(x):
        return ""
    return "".join(ch for ch in str(x) if ch.isdigit())

l_doc = cand[texto_l].map(only_digits)
r_doc = cand[texto_r].map(only_digits)

txt_ok = (l_doc == r_doc) & (r_doc != "")

# --- 4) tolerancia porcentual ---
den = cand[valor_l].abs()
diff = (cand[valor_l] - cand[valor_r]).abs()
tol_ok = np.where(den.eq(0), diff.eq(0), diff <= (pct * den))

cand = cand[txt_ok & tol_ok].copy()

# --- 5) score: menor diferencia porcentual = mejor ---
cand["diff_abs"] = diff[txt_ok & tol_ok]
cand["diff_pct"] = np.where(den[txt_ok & tol_ok].eq(0), 0, cand["diff_abs"] / den[txt_ok & tol_ok])

# --- 6) resolver colisiones para que sea 1-1 ---
# Paso A: cada L se queda con su mejor R
cand = cand.sort_values(["_id_l", "diff_pct"]).drop_duplicates("_id_l", keep="first")

# Paso B: si varios L apuntan al mismo R, nos quedamos con el mejor (y descartamos el resto)
cand = cand.sort_values(["_id_r", "diff_pct"]).drop_duplicates("_id_r", keep="first")

# (Opcional) Paso C: volver a asegurar unicidad por L (por si el paso B dejó huecos)
cand = cand.sort_values(["_id_l", "diff_pct"]).drop_duplicates("_id_l", keep="first")

out = cand.drop(columns=["diff_abs", "diff_pct"])

In [46]:
out.shape

(804, 54)

In [48]:
out['nombre_entidad'].unique()

array(['ICBF REGIONAL AMAZONAS', 'ICBF REGIONAL ANTIOQUIA',
       'ICBF REGIONAL ARAUCA *', 'ICBF REGIONAL ATLANTICO',
       'ICBF REGIONAL BOGOTA',
       'INSTITUTO COLOMBIANO DE BIENESTAR FAMILIAR REGIONAL BOLIVAR',
       'INSTITUTO COLOMBIANO DE BIENESTAR FAMILIAR REGIONAL BOYACA',
       'ICBF REGIONAL CALDAS', 'ICBF REGIONAL CAQUETA',
       'ICBF REGIONAL CASANARE', 'ICBF REGIONAL CAUCA',
       'ICBF REGIONAL CESAR', 'ICBF REGIONAL CHOCO',
       'ICBF REGIONAL CORDOBA', 'ICBF REGIONAL CUNDINAMARCA',
       'ICBF REGIONAL GUAINIA', 'ICBF REGIONAL GUAVIARE',
       'ICBF REGIONAL HUILA', 'ICBF REGIONAL MAGDALENA',
       'ICBF REGIONAL META', 'ICBF REGIONAL NARIÑO',
       'ICBF REGIONAL NORTE DE SANTANDER', 'ICBF REGIONAL PUTUMAYO',
       'ICBF REGIONAL QUINDIO', 'ICBF RISARALDA//',
       'ICBF REGIONAL SANTANDER', 'ICBF REGIONAL SUCRE',
       'ICBF REGIONAL TOLIMA', 'ICBF REGIONAL VALLE DEL CAUCA',
       'ICBF REGIONAL VICHADA', 'ICBF REGIONAL GUAJIRA'], dtype=object)

In [20]:
secopii.head()

,id_contrato,fecha_de_firma,tipo_de_contrato,modalidad_de_contratacion,estado_contrato,objeto_del_contrato,tipodocproveedor,documento_proveedor,proveedor_adjudicado,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,nombre_representante_legal,nombre_entidad,nit_entidad,departamento,ciudad,valor_del_contrato,urlproceso
0,CO1.PCCNTR.7611750,2025-03-06,Otro,Contratación régimen especial,Modificado,COMPLEMENTAR LA ATENCIÓN DE NIÑOS Y NIÑAS A TR...,NIT,800178314,Junta de Accion Comunal Barrio Villa de Guadal...,Sin Descripcion,Sin Descripcion,Flor Maria Hernandez Gallego,ICBF REGIONAL ANTIOQUIA,899999239,Antioquia,Medellín,380973383,https://community.secop.gov.co/Public/Tenderin...
1,CO1.PCCNTR.8737830,2025-12-28,Otro,Contratación régimen especial,En ejecución,PRESTAR LOS SERVICIOS DE EDUCACIÓN INICIAL EN ...,NIT,800014924,Hogar Infantil la ronda,Sin Descripcion,Sin Descripcion,Asociación padres de familia de los niños usu...,ICBF REGIONAL ANTIOQUIA,899999239,Antioquia,Medellín,288200024,https://community.secop.gov.co/Public/Tenderin...
3,CO1.PCCNTR.7879944,2025-05-16,Comodato,Contratación directa,Modificado,ENTREGAR POR PARTE DEL COMODANTE DE FORMA REAL...,NIT,800125258,Asociación de padres de familia de los niños u...,Cédula de Ciudadanía,800125258,Jesús Andrés Ochoa Espinosa,ICBF REGIONAL ANTIOQUIA,899999239,Antioquia,Medellín,0,https://community.secop.gov.co/Public/Tenderin...
5,CO1.PCCNTR.7473183,2025-02-12,Otro,Contratación régimen especial,Modificado,PRESTAR LOS SERVICIOS DE EDUCACIÓN INICIAL EN ...,NIT,900208749,APADESO CONSTRUYENDO FUTURO,Cédula de Ciudadanía,43998860,NATALIA OSORIO BUILES,ICBF REGIONAL ANTIOQUIA,899999239,Antioquia,Medellín,1756209032,https://community.secop.gov.co/Public/Tenderin...
6,CO1.PCCNTR.6002711,2024-02-25,Otro,Contratación régimen especial,terminado,PRESTAR LOS SERVICIOS DE EDUCACIÓN INICIAL EN ...,NIT,800136782,Asociación Social Popular,NIT,8001367828,MARTHA CECILIA AMAYA RESTREPO,ICBF REGIONAL ANTIOQUIA,899999239,Antioquia,Medellín,3838963645,https://community.secop.gov.co/Public/Tenderin...
